<font color="darkred">

# Example 2-1: Classifying movie review

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
from keras import optimizers

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image

### The IMDB dataset


- A set of 50,000 highly-polarized reviews from the Internet Movie Database
    - 25,000 reviews for training and 25,000 reviews for testing
    - 50% negative(label=0) and 50% positive(label=1) reviews



### Loading data

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)


- `num_words=10000`: keep the top 10,000 most frequently occurring words in the training data


In [ ]:
len(train_data)

In [ ]:
train_data[1]

- 각 리뷰를 구성하는 단어 인덱스 리스트 

In [ ]:
train_labels[0]

- 각 리뷰를 긍정=1 또는 부정=0 으로 나타내는 리스트 

In [ ]:
len(train_data)

In [ ]:
max([max(sequence) for sequence in train_data])

- 가장 자주 등장하는 단어 10000개로 제한했기 때문에 단어 인덱스는 9999가 최대값 

In [ ]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

- `word_index`: 단어를 key로, 정수 인덱스를 value로 매칭하는 딕셔너리 
- `reverse_word_index`: 정수 인덱스를 key로, 단어를 value로 매칭하는 딕셔너리 

In [ ]:
word_index['the']

In [ ]:
reverse_word_index[1]

In [ ]:
reverse_word_index.get()

In [ ]:
train_data[0]

In [ ]:
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i-3 , '?') for i in train_data[0]])

- 0번째 리뷰를 문장으로 복원 
- 0="padding", 1="start of sequence", 2="unknown"을 위한 인덱스로 사용하고 있으므로 3을 빼서 단어로 복원 
- 참고: https://keras.io/datasets/

In [ ]:
decoded_review

### Preparing the data
- 한 문장을 $10000\times 1$ vector로 변환
    - 문장이 포함하고 있는 단어 자리는 1, 포함하지 않은 단어 자리는 0으로 채워진 벡터로 변환
    

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

In [ ]:
# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

In [ ]:
x_train.shape

In [ ]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
y_train

In [ ]:
np.shape(x_train), np.shape(y_train)

### Building our network




![3-layer network](https://s3.amazonaws.com/book.keras.io/img/ch3/3_layer_network.png)

In [ ]:
K.clear_session() 

model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(10000,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

- Input은 10000개 단어에 대한 one-hot vector로 표현된 단어들 
- 16개 hidden unit이 있는 layer 2개로 구성 
- 각 layer에서 activation funtion $\sigma(\cdot)$을 ReLU 함수를 사용 

<img src="figures/relu.PNG" width=350>
- Output은 긍정의 확률을 나타내는 하나의 숫자. Sigmoid 함수 사용 
<img src="figures/sigmoid.PNG" width=350>


In [ ]:
model.compile(optimizer=optimizers.adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


- Loss function:`binary_crossentropy` for binary classification 
- Optimizer:  `rmsprop` 

### Validating our approach

Create a "validation set" by setting apart 10,000 samples from the original training data

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

- `batch_size=512`: 512개의 샘플씩 미니 배치를 만들어 gradient를 업데이트 
- `epochs=20`: 모든 샘플에 대해 20번 반복 
- `validation_data=(x_val, y_val)`: 주어진 validation set에 대해 검증 데이터 전달 

In [ ]:
history_dict = history.history
history_dict.keys()

- `model.fit`은 history 객체를 반환. 훈련동안의 모든 정보를 담고 있는 딕셔너리 

In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

- 학습이 진행되면서 training loss는 점차 감소 
- 학습이 진행되면서 validation loss는 감소하다가 4번째 epoch 이후 증가
    - training data에 대해서는 잘 작동하지만 처음 보는 데이터에는 잘 작동하지 않음: overfitting 

In [ ]:

plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Using a trained network to generate predictions on new data


- Overfitting이 발생하기 직전까지 4번의 epoch만 적합시킨 모형으로 test set을 예측하는데 사용 

In [ ]:
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(10000,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.adam(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results

In [ ]:
predictions = model.predict(x_test)

- `x_test`에 대해 모형을 통해 예측한 값을 저장 

In [ ]:
predictions[0]

<font color="darkgreen">

__NOTE__
- 위의 예제에서는 20번의 epoch를 먼저 학습시켜 overfitting 여부를 체크한 후 다시 한번 모형을 적절한 epoch 만큼 학습시키는 과정을 진행했음
- 하지만 학습시간이 오래걸리는 문제에서는 이 방식이 비효율적임
- tensorboard를 활용한다면 학습하면서 동시에 overfitting을 체크하며 model tuning 과정을 보다 효율적으로 진행할 수 있음


References
- [Deep Learning with Python, François Chollet,](https://www.manning.com/books/deep-learning-with-python)